In [11]:
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime
import pandas as pd

# Define start and end dates
start_date = "2010-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')
# Step 1: Fetch NASDAQ data using QQQ ETF as a proxy
try:
    nasdaq_data = yf.download("NDAQ", start=start_date, end=end_date)[["Open", "High", "Low", "Close", "Volume"]]
    print("NASDAQ data sample:")
    print(nasdaq_data.head())
except Exception as e:
    print(f"Error retrieving NASDAQ data: {e}")
    nasdaq_data = pd.DataFrame()  # Empty DataFrame if not available

# Step 2: Fetch economic indicators from FRED
indicators = {
    "InterestRate": "FEDFUNDS",      # Federal Funds Rate
    "ExchangeRate": "DEXUSEU",       # USD to EUR Exchange Rate
    "VIX": "VIXCLS",                 # Volatility Index
    "TEDSpread": "TEDRATE",          # TED Spread
    "EFFR": "EFFR"                   # Effective Federal Funds Rate
}

# Retrieve each indicator and store in a dictionary
econ_data = {}
for label, fred_code in indicators.items():
    try:
        data = pdr.get_data_fred(fred_code, start=start_date, end=end_date)
        print(f"{label} data sample:")
        print(data.head())
        econ_data[label] = data
    except Exception as e:
        print(f"Could not retrieve {label} from FRED: {e}")
        econ_data[label] = pd.DataFrame()  # Empty DataFrame if not available

# Step 3: Fetch Gold and Oil prices using Yahoo Finance
try:
    gold_data = yf.download("GC=F", start=start_date, end=end_date)[["Close"]].rename(columns={"Close": "Gold"})
    print("Gold data sample:")
    print(gold_data.head())
except Exception as e:
    print(f"Error retrieving Gold data: {e}")
    gold_data = pd.DataFrame()

try:
    oil_data = yf.download("CL=F", start=start_date, end=end_date)[["Close"]].rename(columns={"Close": "Oil"})
    print("Oil data sample:")
    print(oil_data.head())
except Exception as e:
    print(f"Error retrieving Oil data: {e}")
    oil_data = pd.DataFrame()

# Step 4: Combine all datasets
# Convert economic indicators dictionary to a DataFrame, excluding any that failed
econ_df = pd.concat([df for df in econ_data.values() if not df.empty], axis=1)
econ_df.columns = [label for label, df in econ_data.items() if not df.empty]

# Combine all data into a single DataFrame
combined_data = nasdaq_data.join(econ_df, how='outer')
combined_data = combined_data.join(gold_data, how='outer')
combined_data = combined_data.join(oil_data, how='outer')

# Fill missing values and save to CSV if any data exists
if not combined_data.empty:
    combined_data = combined_data.fillna(method='ffill').dropna()
    print("Data has been successfully saved to 'nasdaq_combined_data.csv'.")
else:
    print("No data to save. Check each data source for availability.")


[*********************100%***********************]  1 of 1 completed


NASDAQ data sample:
                 Open       High        Low      Close   Volume
Date                                                           
2020-01-02  35.753334  35.986668  35.723331  35.939999  1923900
2020-01-03  35.660000  36.160000  35.580002  36.093334  2164800
2020-01-06  35.930000  36.086666  35.630001  36.070000  2155200
2020-01-07  36.000000  36.086666  35.463333  35.473331  1673100
2020-01-08  35.503334  35.653332  35.080002  35.276669  3501000
InterestRate data sample:
            FEDFUNDS
DATE                
2020-01-01      1.55
2020-02-01      1.58
2020-03-01      0.65
2020-04-01      0.05
2020-05-01      0.05
ExchangeRate data sample:
            DEXUSEU
DATE               
2020-01-01      NaN
2020-01-02   1.1166
2020-01-03   1.1173
2020-01-06   1.1187
2020-01-07   1.1138
VIX data sample:
            VIXCLS
DATE              
2020-01-01     NaN
2020-01-02   12.47
2020-01-03   14.02
2020-01-06   13.85
2020-01-07   13.79
TEDSpread data sample:
            TEDRATE


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

EFFR data sample:
            EFFR
DATE            
2020-01-01   NaN
2020-01-02  1.55
2020-01-03  1.55
2020-01-06  1.55
2020-01-07  1.55
Gold data sample:
                   Gold
Date                   
2020-01-02  1524.500000
2020-01-03  1549.199951
2020-01-06  1566.199951
2020-01-07  1571.800049
2020-01-08  1557.400024
Oil data sample:
                  Oil
Date                 
2020-01-02  61.180000
2020-01-03  63.049999
2020-01-06  63.270000
2020-01-07  62.700001
2020-01-08  59.610001
Data has been successfully saved to 'nasdaq_combined_data.csv'.


In [12]:
df=combined_data

In [13]:
df = df.rename(columns={"Unnamed: 0": "Date"})


In [14]:
df.head()

,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil
2020-01-02,35.753334,35.986668,35.723331,35.939999,1923900.0,1.55,1.1166,12.47,0.39,1.55,1524.500000,61.180000
2020-01-03,35.660000,36.160000,35.580002,36.093334,2164800.0,1.55,1.1173,14.02,0.38,1.55,1549.199951,63.049999
2020-01-06,35.930000,36.086666,35.630001,36.070000,2155200.0,1.55,1.1187,13.85,0.34,1.55,1566.199951,63.270000
2020-01-07,36.000000,36.086666,35.463333,35.473331,1673100.0,1.55,1.1138,13.79,0.37,1.55,1571.800049,62.700001
2020-01-08,35.503334,35.653332,35.080002,35.276669,3501000.0,1.55,1.1117,13.45,0.32,1.55,1557.400024,59.610001


In [15]:
df.tail()

,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil
2024-10-21,74.699997,74.870003,74.050003,74.300003,1902500.0,5.13,1.0854,18.37,0.09,4.83,2723.100098,70.559998
2024-10-22,73.730003,74.459999,73.529999,74.290001,2365000.0,5.13,1.0854,18.20,0.09,4.83,2744.199951,72.089996
2024-10-23,73.980003,74.489998,73.669998,73.910004,2513600.0,5.13,1.0854,19.24,0.09,4.83,2714.399902,70.769997
2024-10-24,73.239998,75.580002,72.800003,75.239998,4038400.0,5.13,1.0854,19.08,0.09,4.83,2734.899902,70.190002
2024-10-25,75.599998,75.680000,74.029999,74.690002,2230100.0,5.13,1.0854,19.08,0.09,4.83,2760.800049,71.690002


In [16]:
# Assign a name to the index
df.index.name = "Date"


In [17]:
df.to_csv("nasdq.csv")

In [20]:
df.isnull().sum()

Open            0
High            0
Low             0
Close           0
Volume          0
InterestRate    0
ExchangeRate    0
VIX             0
TEDSpread       0
EFFR            0
Gold            0
Oil             0
dtype: int64